In [1]:
#import dependencies
#open chromedriver

from splinter import Browser
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pymongo
from selenium import webdriver
import os
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import selenium

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
browser = webdriver.Chrome('/Python38/chromedriver')

In [3]:
browser.get('https://s3.amazonaws.com/tripdata/index.html')

In [4]:
#get all links from browser
all_links = [link.get_attribute('href') for link in browser.find_elements_by_tag_name('a')]

In [5]:
#extract necessary links to clean

#linksJC_19 = [link for link in all_links if 'JC' in link if '2019' in link]
linksJC_20 = [link for link in all_links if 'JC' in link if '2020' in link]
#links_19 = [link for link in all_links if 'JC' not in link if '2019' in link]
#links_20 = [link for link in all_links if 'JC' not in link if '2020' in link]

In [24]:
# function for downloading, unziping and dataframing data from the links
def create_df(link):
    
    url = urllib.request.urlopen(link)
    output = open('temporary.zip', 'wb')    
    output.write(url.read())
    output.close()
    dataframe = pd.read_csv('temporary.zip')
    
    if (len(dataframe.columns) == 15):
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender']
    else:
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID',
       'Localized Value', 'User Type', 'Birth Year', 'Gender']
        dataframe = dataframe.drop('Localized Value',1)
        
    print(link)
    os.remove('temporary.zip')
    
    return dataframe

In [25]:
#function for cleaning and prep of df
def prepare_df(df):
    
#drop n/a
    df = df.dropna(how='any').reset_index(drop=True)
    
#change data types
    #df['Birth_Year'] = df.Birth_Year.astype(int)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['Stop Time'] = pd.to_datetime(df['Stop Time'])
    
#add age column
    df['Age'] = 2020 - df['Birth Year']
    
#exclude ages > 90 years
    df = df[df['Age'] < 90]
    
    return df
    
    

In [26]:
# create empty dfs
#ny19 = pd.DataFrame()
ny20 = pd.DataFrame()

In [27]:
#append to new df
#for link in links_19:
    #temporary_df = create_df(link)
    #ny19 = ny19.append(temporary_df, ignore_index=True, sort=False)
    

In [28]:
#append to new df
for link in linksJC_20:
    temporary_df = create_df(link)
    ny20 = ny20.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/JC-202001-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202002-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202003-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202004-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202005-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202006-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202007-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202008-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202009-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-202010-citibike-tripdata.csv.zip


ValueError: Multiple files found in compressed zip file ['JC-202011-citibike-tripdata.csv', '__MACOSX/._JC-202011-citibike-tripdata.csv']

In [29]:
#clean and prepare df
#ny19 = prepare_df(ny19)
ny20 = prepare_df(ny20)

In [30]:
#ny19.head()

In [47]:
ny20.head()

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Age
0,226,2020-01-01 00:04:50.192,2020-01-01 00:08:37.037,3186,Grove St PATH,40.719586,-74.043117,3211,Newark Ave,40.721525,-74.046305,29444,Subscriber,1984,2,36
1,377,2020-01-01 00:16:01.670,2020-01-01 00:22:19.080,3186,Grove St PATH,40.719586,-74.043117,3269,Brunswick & 6th,40.726012,-74.050389,26305,Subscriber,1989,2,31
2,288,2020-01-01 00:17:33.877,2020-01-01 00:22:22.442,3186,Grove St PATH,40.719586,-74.043117,3269,Brunswick & 6th,40.726012,-74.050389,29268,Customer,1989,1,31
3,435,2020-01-01 00:32:05.902,2020-01-01 00:39:21.066,3195,Sip Ave,40.730897,-74.063913,3280,Astor Place,40.719282,-74.071262,29278,Customer,1969,0,51
4,231,2020-01-01 00:46:19.678,2020-01-01 00:50:11.344,3186,Grove St PATH,40.719586,-74.043117,3276,Marin Light Rail,40.714584,-74.042817,29276,Subscriber,1983,2,37


In [32]:
#ny19.dtypes

In [48]:
#ny20.dtypes


In [34]:
#nydf = ny19.append(ny20, ignore_index=True, sort=False)

In [35]:
ny20.count()

Trip Duration (sec)        303808
Start Time                 303808
Stop Time                  303808
Start Station ID           303808
Start Station Name         303808
Start Station Latitude     303808
Start Station Longitude    303808
End Station ID             303808
End Station Name           303808
End Station Latitude       303808
End Station Longitude      303808
Bike ID                    303808
User Type                  303808
Birth Year                 303808
Gender                     303808
Age                        303808
dtype: int64

In [36]:
#load last csv, due to double files in the zip file on the website and after wrestling with it I decided to do it this way in bringing in the last csv. 
JC202011 = "data/JC-202011-citibike-tripdata.csv"
JC202011 = pd.read_csv(JC202011)
JC202011_df = pd.DataFrame(JC202011)
JC202011_df.head(1000000)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,756,2020-11-01 00:11:32.6250,2020-11-01 00:24:09.5850,3275,Columbus Drive,40.718355,-74.038914,3225,Baldwin at Montgomery,40.723659,-74.064194,42436,Subscriber,1984,1
1,584,2020-11-01 00:14:20.4820,2020-11-01 00:24:04.7820,3681,Grand St,40.715178,-74.037683,3209,Brunswick St,40.724176,-74.050656,46532,Customer,1969,0
2,403,2020-11-01 00:22:55.8820,2020-11-01 00:29:39.0950,3203,Hamilton Park,40.727596,-74.044247,3483,Montgomery St,40.719420,-74.050990,35139,Subscriber,1980,1
3,498,2020-11-01 00:41:37.4270,2020-11-01 00:49:55.7010,3268,Lafayette Park,40.713464,-74.062859,3213,Van Vorst Park,40.718489,-74.047727,46539,Customer,1969,0
4,151,2020-11-01 00:53:21.2340,2020-11-01 00:55:52.8060,3185,City Hall,40.717733,-74.043845,3276,Marin Light Rail,40.714584,-74.042817,44388,Subscriber,1986,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21270,849,2020-11-30 23:08:40.1870,2020-11-30 23:22:50.0830,3195,Sip Ave,40.730897,-74.063913,3679,Bergen Ave,40.722104,-74.071455,44742,Subscriber,1996,2
21271,370,2020-11-30 23:09:27.0180,2020-11-30 23:15:37.3330,3195,Sip Ave,40.730897,-74.063913,3679,Bergen Ave,40.722104,-74.071455,17450,Subscriber,1999,1
21272,341,2020-11-30 23:37:53.0580,2020-11-30 23:43:34.1200,3187,Warren St,40.721124,-74.038051,3202,Newport PATH,40.727224,-74.033759,42528,Subscriber,1991,1
21273,5272,2020-11-30 23:49:31.8650,2020-12-01 01:17:24.2540,3679,Bergen Ave,40.722104,-74.071455,3679,Bergen Ave,40.722104,-74.071455,14657,Customer,1990,1


In [37]:
#function for cleaning and prep of df
#def prepare_df(JC202011_df):

#Rename the column headers
JC202011_df = JC202011_df.rename(columns={"tripduration": "Trip Duration (sec)",
                                                         "starttime": "Start Time",
                                                         "stoptime": "Stop Time",
                                                         "start station id": "Start Station ID", 
                                                         "start station name": "Start Station Name", 
                                                         "start station latitude": "Start Station Latitude", 
                                                         "start station longitude": "Start Station Longitude", 
                                                         "end station id": "End Station ID", 
                                                         "end station name": "End Station Name",
                                                         "end station latitude": "End Station Latitude",
                                                         "end station longitude": "End Station Longitude",
                                                         "bikeid": "Bike ID", 
                                                         "usertype": "User Type", 
                                                         "birth year": "Birth Year", 
                                                         "gender": "Gender"})                                          
                                         
                                         
                                         
    
#drop n/a
#JC202011_df = JC202011_df.dropna(how='any').reset_index(drop=True)
    
#change data types
#JC202011_df['Birth Year'] = JC202011_df.Birth Year.astype(int)
JC202011_df['Start Time'] = pd.to_datetime(JC202011_df['Start Time'])
JC202011_df['Stop Time'] = pd.to_datetime(JC202011_df['Stop Time'])
    
#add age column
JC202011_df['Age'] = 2020 - JC202011_df['Birth Year']
    
#exclude ages > 90 years
JC202011_df = JC202011_df[JC202011_df['Age'] < 90]
    
JC202011_df

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Age
0,756,2020-11-01 00:11:32.625,2020-11-01 00:24:09.585,3275,Columbus Drive,40.718355,-74.038914,3225,Baldwin at Montgomery,40.723659,-74.064194,42436,Subscriber,1984,1,36
1,584,2020-11-01 00:14:20.482,2020-11-01 00:24:04.782,3681,Grand St,40.715178,-74.037683,3209,Brunswick St,40.724176,-74.050656,46532,Customer,1969,0,51
2,403,2020-11-01 00:22:55.882,2020-11-01 00:29:39.095,3203,Hamilton Park,40.727596,-74.044247,3483,Montgomery St,40.719420,-74.050990,35139,Subscriber,1980,1,40
3,498,2020-11-01 00:41:37.427,2020-11-01 00:49:55.701,3268,Lafayette Park,40.713464,-74.062859,3213,Van Vorst Park,40.718489,-74.047727,46539,Customer,1969,0,51
4,151,2020-11-01 00:53:21.234,2020-11-01 00:55:52.806,3185,City Hall,40.717733,-74.043845,3276,Marin Light Rail,40.714584,-74.042817,44388,Subscriber,1986,1,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21270,849,2020-11-30 23:08:40.187,2020-11-30 23:22:50.083,3195,Sip Ave,40.730897,-74.063913,3679,Bergen Ave,40.722104,-74.071455,44742,Subscriber,1996,2,24
21271,370,2020-11-30 23:09:27.018,2020-11-30 23:15:37.333,3195,Sip Ave,40.730897,-74.063913,3679,Bergen Ave,40.722104,-74.071455,17450,Subscriber,1999,1,21
21272,341,2020-11-30 23:37:53.058,2020-11-30 23:43:34.120,3187,Warren St,40.721124,-74.038051,3202,Newport PATH,40.727224,-74.033759,42528,Subscriber,1991,1,29
21273,5272,2020-11-30 23:49:31.865,2020-12-01 01:17:24.254,3679,Bergen Ave,40.722104,-74.071455,3679,Bergen Ave,40.722104,-74.071455,14657,Customer,1990,1,30


In [49]:
#JC202011_df.dtypes

In [39]:
JC202011_df.count()

Trip Duration (sec)        21275
Start Time                 21275
Stop Time                  21275
Start Station ID           21275
Start Station Name         21275
Start Station Latitude     21275
Start Station Longitude    21275
End Station ID             21275
End Station Name           21275
End Station Latitude       21275
End Station Longitude      21275
Bike ID                    21275
User Type                  21275
Birth Year                 21275
Gender                     21275
Age                        21275
dtype: int64

In [40]:
nydf = ny20.append(JC202011_df, ignore_index=True, sort=False)

#nydf = nydf.reset_index(drop=True)

In [41]:
nydf

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Age
0,226,2020-01-01 00:04:50.192,2020-01-01 00:08:37.037,3186,Grove St PATH,40.719586,-74.043117,3211,Newark Ave,40.721525,-74.046305,29444,Subscriber,1984,2,36
1,377,2020-01-01 00:16:01.670,2020-01-01 00:22:19.080,3186,Grove St PATH,40.719586,-74.043117,3269,Brunswick & 6th,40.726012,-74.050389,26305,Subscriber,1989,2,31
2,288,2020-01-01 00:17:33.877,2020-01-01 00:22:22.442,3186,Grove St PATH,40.719586,-74.043117,3269,Brunswick & 6th,40.726012,-74.050389,29268,Customer,1989,1,31
3,435,2020-01-01 00:32:05.902,2020-01-01 00:39:21.066,3195,Sip Ave,40.730897,-74.063913,3280,Astor Place,40.719282,-74.071262,29278,Customer,1969,0,51
4,231,2020-01-01 00:46:19.678,2020-01-01 00:50:11.344,3186,Grove St PATH,40.719586,-74.043117,3276,Marin Light Rail,40.714584,-74.042817,29276,Subscriber,1983,2,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325078,849,2020-11-30 23:08:40.187,2020-11-30 23:22:50.083,3195,Sip Ave,40.730897,-74.063913,3679,Bergen Ave,40.722104,-74.071455,44742,Subscriber,1996,2,24
325079,370,2020-11-30 23:09:27.018,2020-11-30 23:15:37.333,3195,Sip Ave,40.730897,-74.063913,3679,Bergen Ave,40.722104,-74.071455,17450,Subscriber,1999,1,21
325080,341,2020-11-30 23:37:53.058,2020-11-30 23:43:34.120,3187,Warren St,40.721124,-74.038051,3202,Newport PATH,40.727224,-74.033759,42528,Subscriber,1991,1,29
325081,5272,2020-11-30 23:49:31.865,2020-12-01 01:17:24.254,3679,Bergen Ave,40.722104,-74.071455,3679,Bergen Ave,40.722104,-74.071455,14657,Customer,1990,1,30


In [50]:
#nydf.count()
#nydf.dtypes

In [51]:
nydf.to_csv('jcbike_from_url20.csv', encoding='utf-8', index=False)